In [1]:
import pandas as pd

In [2]:
#TODO 1: Read all the data from bc.csv, how many rows is there ? Columns ?

In [2]:
df = pd.read_csv("bc.csv")

In [3]:
df

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,NaN
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,NaN
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,NaN
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,NaN


In [4]:
#TODO 2: Insert the bc.csv in the sqlite format
import tqdm
import sqlite3

In [5]:
conn = sqlite3.connect('bc.db')
c = conn.cursor()


In [6]:
df.to_sql('MaTable', conn, if_exists='replace', index=False)

569

In [7]:
conn.close()

In [9]:
#TODO 3: In the same db create a new table called tomatch with two variables id_ and rdm_float.

In [8]:
conn = sqlite3.connect('bc.db')
c = conn.cursor()
c.execute('''CREATE TABLE tomatch
             (id_ INTEGER PRIMARY KEY,
              rdm_float FLOAT)''')
conn.commit()
conn.close()

In [ ]:
#TODO 4: In the table tomatch insert observations where the id_ starts from 50k, 
#ends at 10000000 and is only even numbers. rdm_float is a float random for each observation

In [9]:
import random

In [10]:
conn = sqlite3.connect('bc.db')

In [11]:
observations = [(i, random.random()) for i in range(50000, 10000001, 2)]

In [12]:
c = conn.cursor()
c.executemany('INSERT INTO tomatch (id_, rdm_float) VALUES (?, ?)', observations)
conn.commit()

In [13]:
#TODO 5: Do an Inner join between the two tables 
#and compute the different aggregate functions on a column of your choice.
c.execute("""SELECT MaTable.id, AVG(MaTable.radius_mean), SUM(MaTable.radius_mean), MAX(MaTable.radius_mean), MIN(MaTable.radius_mean), COUNT(MaTable.radius_mean)
FROM MaTable
INNER JOIN  tomatch  ON MaTable.id = tomatch.id_""")
docs = c.fetchall()
print(docs)

[(862722, 13.997218106995879, 3401.3239999999987, 27.22, 6.981, 243)]


In [14]:
#TODO 6: How many observations have a radius_mean greater than 15 ? 
#From them how many have a texture_mean greater than 20 ?
c.execute("""select count(id)
FROM MaTable
WHERE radius_mean >15 and texture_mean > 20 """ )
docs = c.fetchall()
print(docs)

[(106,)]


In [15]:
#TODO 7: For each category in diagnosis what is the mean of all the variables ? 
#Compute the difference between the mean of both groups ( idée)
c.execute("""SELECT AVG(id), AVG(radius_mean), AVG(texture_mean), AVG(perimeter_mean)
FROM MaTable
WHERE diagnosis ='M'
""")
docs = c.fetchall()
print(docs)

[(36818050.443396226, 17.46283018867925, 21.60490566037735, 115.36537735849062)]


In [16]:
#TODO 8: Change the name of the column "diagnosis" to "label"
c.execute("""ALTER TABLE MaTable RENAME COLUMN diagnosis TO Label""")
conn.commit()

In [17]:
conn.close()

In [20]:
conn = sqlite3.connect('bc.db')

In [21]:
#TODO 9: Change the type of the column "area_mean" to int
conn = sqlite3.connect('bc.db')
c.execute("""ALTER TABLE MaTable  ALTER COLUMN area_mean INTEGER """)
conn.commit()


ProgrammingError: Cannot operate on a closed database.

In [22]:
#TODO 10: Add a column named "day" of type DATE.
conn = sqlite3.connect('bc.db')
c = conn.cursor()
c.execute("""ALTER TABLE MaTable ADD day DATE""")
conn.commit()

In [23]:
conn = sqlite3.connect('bc.db')
c.execute('''CREATE TABLE tomatch2 AS
             SELECT id_ + 10 AS id_, rdm_float
             FROM tomatch''')
conn.commit()


In [25]:
#TODO 12: Do an inner join of bc, tomatch and tomatch2 where the radius_mean is greater than 15, 
#and the rdm_float is greater than 0.50 in tomatch AND tomatch2.
c.execute('''SELECT *
FROM MaTable
INNER JOIN tomatch ON MaTable.id = tomatch.id_
inner JOIN tomatch2 on MaTable.id = tomatch2.id_
where  MaTable.radius_mean > 15
And (tomatch.rdm_float >0.5) and (tomatch2.rdm_float > 0.5)
''')
docs = c.fetchall()
print(docs)

[(89122, 'M', 19.4, 18.18, 127.2, 1145.0, 0.1037, 0.1442, 0.1626, 0.09464, 0.1893, 0.05892, 0.4709, 0.9951, 2.903, 53.16, 0.005654, 0.02199, 0.03059, 0.01499, 0.01623, 0.001965, 23.79, 28.65, 152.4, 1628.0, 0.1518, 0.3749, 0.4316, 0.2252, 0.359, 0.07787, None, None, 89122, 0.5159286301514808, 89122, 0.596577962277758), (90312, 'M', 19.55, 23.21, 128.9, 1174.0, 0.101, 0.1318, 0.1856, 0.1021, 0.1989, 0.05884, 0.6107, 2.836, 5.383, 70.1, 0.01124, 0.04097, 0.07469, 0.03441, 0.02768, 0.00624, 20.82, 30.44, 142.0, 1313.0, 0.1251, 0.2414, 0.3829, 0.1825, 0.2576, 0.07602, None, None, 90312, 0.7621368498532403, 90312, 0.5384453399734267), (849014, 'M', 19.81, 22.15, 130.0, 1260.0, 0.09831, 0.1027, 0.1479, 0.09498, 0.1582, 0.05395, 0.7582, 1.017, 5.865, 112.4, 0.006494, 0.01893, 0.03391, 0.01521, 0.01356, 0.001997, 27.32, 30.88, 186.8, 2398.0, 0.1512, 0.315, 0.5372, 0.2388, 0.2768, 0.07615, None, None, 849014, 0.870928759987474, 849014, 0.8055386134934017), (852552, 'M', 16.65, 21.38, 110.0, 904